# Generic styles with generic lambdas

Styling graphics object is a common task when plotting results. Having a common style function can therefore reduce code significantly, particularly when having many objects which need to be styled. However often it appears that macros contain both histograms and graphs. Due to multiple inheritance histograms and graphs have no common base clase which can be used when styling objects. In case the style function would use TObject as common base class this would result in downcasting within the style function, which leads to code duplication. To overcome this, templates (since ROOT6) can be used. Another possibility is to use generic lambdas and closures (requires c++14).

In [ ]:
// Defining test data: 1 TGraph object, 1 TH1D object
std::vector<double> xvalsGraph = {0.5, 1.5, 2.5, 3.5, 4.5},
                    exvalsGraph = {0.5, 0.5, 0.5, 0.5, 0.5},
                    yvalsGraph = {0., 3., 10., 5., 2.},
                    eyvalsGraph = {0., 0.1, 0.2, 0.15, 0.05};
TGraphErrors *testgraph = new TGraphErrors;
for(auto i : ROOT::TSeqI(0, 6)) {
    testgraph->SetPoint(i, xvalsGraph[i], yvalsGraph[i]);
    testgraph->SetPointError(i, exvalsGraph[i], eyvalsGraph[i]);
}
TH1 *testhisto = new TH1D("testhisto", "Test histogram", 5, 0., 5.);
for(auto i : ROOT::TSeqI(0., 30)) testhisto->Fill(gRandom->Gaus(2.5, 1.));

## Closure for style

Create a closure for the graphics style. Don't forget: The outer function takes the parameters for color and marker, the inner function captures those and takes as argument the objects to be styled.

In [ ]:
// Your code here
auto style = [](Color_t col, Style_t mrk){
    return [col, mrk](auto graphicsobject) {
        graphicsobject->SetMarkerColor(col);
        graphicsobject->SetMarkerStyle(mrk);
        graphicsobject->SetLineColor(col);
    };
};

Create a style for the graph with color red and marker 24, and one for the histogram with color blue and marker 25. In real life it can happen that one uses a lot more styles (i.e. for different conditions), therefore storing them in vectors or maps makes a lot of sense.

In [ ]:
// Your code here
auto graphstyle = style(kRed, 24), histstyle = style(kBlue, 25);

## Create your final plot
Apply the styles you created before to the graph and the histogram, and plot them. What would happen if you swap the styles (use the style for the histogram for the graph and vice versa)?

In [ ]:
// your code here
histstyle(testhisto);
graphstyle(testgraph);

auto plot = new TCanvas("plot", "plot", 640, 480);
auto frame = new TH1F("frame", "; x-values; entries", 10, 0., 5.);
frame->SetDirectory(nullptr);
frame->SetStats(false);
frame->GetYaxis()->SetRangeUser(0., 20.);
frame->Draw("axis");
testgraph->Draw("epsame");
testhisto->Draw("epsame");
plot->Draw();